In [4]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import matplotlib.pyplot as plt
import joblib
import requests
import json

# loading our dataset with simulated user_id and 10000 new generated transactions(synthetically)
data = pd.read_csv('/kaggle/input/personal-transactions-userid-new-transactions/aug_personal_transactions_with_UserId.csv')

# encoding our categorical variables
data_encoded = pd.get_dummies(data[['User ID', 'Category', 'Account Name', 'Transaction Type']])

# definition of features (X) and target (y) for user expense classification
X_classification = data_encoded[data_encoded['Transaction Type_debit'] == 1].drop(columns=['Transaction Type_debit', 'Transaction Type_credit'])
y_classification = data[data['Transaction Type'] == 'debit']['Amount']

# creating three classes for expense classification: low cost, medium cost and high cost
def classify_expense(amount):
    if amount < 30:
        return 'low cost'
    elif amount < 60:
        return 'medium cost'
    else:
        return 'high cost'

# applying label encoding to the target variable
y_classification = y_classification.apply(classify_expense)
class_mapping = {'low cost': 0, 'medium cost': 1, 'high cost': 2}
y_classification_encoded = y_classification.map(class_mapping)

# splitting the data into training and testing sets for classification
X_train_classification, X_test_classification, y_train_classification, y_test_classification = train_test_split(X_classification, y_classification_encoded, test_size=0.2, random_state=42)

# creating an XGBoost Classifier and training it(fitting) for expense classification
classification_model = xgb.XGBClassifier(objective="multi:softmax", num_class=3, random_state=42)
classification_model.fit(X_train_classification, y_train_classification)

# make predictions on the test data 
y_pred_classification = classification_model.predict(X_test_classification)

#function to classify the expenses based on the amount's magnitude
def expense_level_based_classifier(user_id):
    
     # user_specific data for expense prediction
        user_data = data_encoded[data_encoded['User ID'] == user_id].drop(columns=['User ID'])
        
        if not user_data.empty:
            # ensuring that user_data has the same columns as X_train_classification
            missing_columns = set(X_train_classification.columns) - set(user_data.columns)
            for column in missing_columns:
                user_data[column] = 0  # filling missing columns with default values (0)
            
            # re_order the columns to match the order in X_train_classification
            user_data = user_data[X_train_classification.columns]
            
            # predict if the user's transaction is an expense or income
            expense_classification = classification_model.predict(user_data)
            return expense_classification
            
        else:
            return None


# function to make personalized budget recommendations using hybrid recommendation
def hybrid_recommendation(user_id):
    try:
        # user_specific data for expense prediction
        user_data = data_encoded[data_encoded['User ID'] == user_id].drop(columns=['User ID'])
        
        if not user_data.empty:
            # ensuring that user_data has the same columns as X_train_classification
            missing_columns = set(X_train_classification.columns) - set(user_data.columns)
            for column in missing_columns:
                user_data[column] = 0  # filling missing columns with default values (0)
            
            # re_order the columns to match the order in X_train_classification
            user_data = user_data[X_train_classification.columns]
            
            # predict if the user's transaction is an expense or income
            is_expense = classification_model.predict(user_data)
            
            # check if all elements in is_expense are True 
            if all(is_expense):
                if 'Transaction Type_credit' in user_data.columns:
                    user_data = user_data.drop(columns=['Transaction Type_credit'])
                recommendation = collaborative_model.predict(user_data)
            else:
                recommendation = 0  # Assuming 0 for income transactions
            return recommendation[0]
        else:
            return None
    except KeyError:
        return None

# initialize collaborative recommendation model
collaborative_model = xgb.XGBRegressor(objective="reg:squarederror", random_state=42)

# define features (X) and target (y) for collaborative recommendation
X_collaborative = data_encoded[data_encoded['Transaction Type_debit'] == 1].drop(columns=['Transaction Type_debit', 'Transaction Type_credit'])
y_collaborative = data[data['Transaction Type'] == 'debit']['Amount']

# split the data into training and testing sets for collaborative recommendation
X_train_collaborative, X_test_collaborative, y_train_collaborative, y_test_collaborative = train_test_split(X_collaborative, y_collaborative, test_size=0.2, random_state=42)

# Train the collaborative recommendation model
collaborative_model.fit(X_train_collaborative, y_train_collaborative)

# Function to generate financial advice using Open AI's GPT-3.5 Turbo
def generate_financial_advice(user_id,hybrid_recommendation):
    # Open AI API key
    openai_api_key = 'private_token'
    
    if hybrid_recommendation is not None:
        user_message = f"User {user_id} should consider a budget of ${hybrid_recommendation:.2f} for their expenses."
    else:
        user_message = f"User {user_id} has no expense data available."
    
    prompt = f"Provide financial advice for User {user_id} to help them manage their budget effectively. Based on their financial situation, it is recommended that they allocate ${budget_recommendation:.2f} for their monthly expenses. Please offer advice on saving, investing, or any financial strategies to improve their financial well-being. Keep it under 4 points."

    
    openai_endpoint = "https://api.openai.com/v1/chat/completions"
    headers = {
        "Content-Type": "application/json",
        "Authorization":"Bearer {}".format(openai_api_key)
    }
    
    data = {
                "model": "gpt-3.5-turbo", 
                "messages": [
                                {
                                    "role": "user",
                                    "content": prompt
                                }
                            ]
            }
    
    
    response = requests.post(openai_endpoint, headers=headers, json=data)
    response_data = json.loads(response.text)
    
    advice = response_data["choices"][0]["message"]["content"]
    return advice 


# For testing 
'''
user_id = 1
budget_recommendation = hybrid_recommendation(user_id)
financial_advice = generate_financial_advice(user_id, budget_recommendation)

print(expense_level_based_classifier(user_id))

if budget_recommendation is not None:
    print(f"Personalized Budget Recommendation for User {user_id}: ${budget_recommendation:.2f}")
    print("Financial Advice:")
    print(financial_advice)
else:
    print(f"No budget recommendation available for User {user_id}.")
'''

[2 1 2 2 1 1 2 2 2 2 2 2 2 1 2 1 2 1 2 2 2 2 2 2 1 2 1 2 1 2 2 2 2 2 2 1 2
 2 2 1 1 2 2 2 2 2 2 1 2 2 2 1 1 2 2 2 2 2 2 2 1 2 2 1 1 2 2 2 2 2 2 2 2 2
 1 2 1 1 2 2 2 2 2 2 2 1 2 1 2 1 2 2 2 2 2 2 2 2 1 2 2 1 1 2 2 2 2 2 2 2 1
 2 2 1 1 2 2 2 2 2 2 1 2 2 1 1 2 2 2 2 2 1 2 2 1 1 2 2 2 2 2 2 2 2 1 2 2 2
 1 1 2 2 2 2 2 2 2 1 2 2 2 2 1 2 1 2 2 2 2 2 2 1 2 2 2 2 1 1 2 2 2 2 2 1 2
 2 2 1 1 2 2 2 2 2 2 2 1 2 2 2 1 1 2 2 2 2 2 2 2 2 1 2 2 1 1 2 2 2 2 2 2 2
 2 2 2 2 1 1 1 2 2 2 2 2 2 2 2 2 2 1 2 2 1 2 1 2 2 2 2 2 2 2 1 2 2 2 2 1 1
 2 2 2 2 2]
Personalized Budget Recommendation for User 1: $1178.78
Financial Advice:
1. Create a budget and track your expenses: Start by listing all your monthly expenses and income. Categorize your expenses into fixed (rent, bills, loan payments) and variable (groceries, entertainment). Monitor your spending habits regularly to identify areas where you can cut back and save money.

2. Build an emergency fund: Set aside a portion of your income each month for unexpecte